# RNN-модель для генерации текста с помощью PyTorch

## Импорты

In [72]:
import torch
import numpy as np
import pandas as pd
from torch import nn, optim
from torch.utils.data import DataLoader
from collections import Counter

## Загрузка и предобработка данных

Для обучения воспользуемся [датасетом](https://www.kaggle.com/datasets/lizakonopelko/disco-elysium-dialogue-texts), состоящим из всего текста игры Disco Elysium

Перед построением модели необходимо провести предварительную обработку данных, избавившись от некоторых особых символов, плохо влияющих на результат генерации.
Также для ускорения дальнейшего обучения и предотвращения возможных проблем с нехваткой памяти будем использовать только первые 1500 записей

In [73]:
train_df = pd.read_csv('/content/texts_extracted.csv', header=None, sep='\t')
train_df.rename(columns={0: "text"}, inplace=True)
train_df['text'] = train_df['text'].str.replace('"', '')
train_df['text'] = train_df['text'].str.replace("'", '')
train_df.head()

,text
0,88... This elevator was maintained a long time...
1,A man my age? What are you implying? Im at the...
2,A science person? He snarls. The *so-called* s...
3,"A slow, sad song started playing. Like organ m..."
4,"After life, death -- after death, life again. ..."


In [74]:
print('Number of entries: ', train_df.shape[0])
print('Number of characters: ', train_df['text'].str.len().sum())

Number of entries:  58818
Number of characters:  4716416


In [75]:
train_df = train_df.sample(n=1500)
print('Number of characters: ', train_df['text'].str.len().sum())

Number of characters:  120573


## Создание модели
Это стандартная модель PyTorch. Embedding слой преобразует индексы слов в векторы слов. LSTM является основной обучаемой частью сети - в реализации PyTorch внутри ячейки LSTM реализован механизм стробирования, позволяющий обучать длинные последовательности данных

LSTM имеет дополнительную информацию о состоянии, которую он переносит между эпизодами обучения. Функция forward имеет аргумент prev_state. Это состояние хранится вне модели и передается вручную

Также имеется функция init_state. Она вызывается в начале каждой эпохи для инициализации нужной формы состояния

In [76]:
class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3

        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim)
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2)
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)

        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

## Класс Dataset
Создаем класс Dataset, наследуемый от класса torch.utils.data.Dataset

Функция load_words загружает датасет. В датасете подсчитываются уникальные слова, определяющие размер словарного запаса сети и размер векторов. index_to_word и word_to_index преобразуют слова в числовые индексы и наоборот.

In [77]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequence_length):
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()
        self.sequence_length = sequence_length
        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def load_words(self):
        source = train_df
        text = train_df['text'].str.cat(sep=' ')
        return text.split(' ')

    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - self.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.sequence_length+1]))

## Обучение модели

In [78]:
n_epochs = 40
batch_size = 128

dataset = Dataset(4) # задаем окно, на основании которого будет генерироваться текст
model = Model(dataset)
dataloader = DataLoader(dataset, batch_size=128, num_workers=2, pin_memory=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

best_model = None
best_loss = np.inf


for epoch in range(n_epochs):
  model.train()
  state_h, state_c = model.init_state(4)

  for batch, (x, y) in enumerate(dataloader):
    y_pred, (state_h, state_c) = model(x, (state_h, state_c))
    loss = loss_fn(y_pred.transpose(1, 2), y)

    state_h = state_h.detach()
    state_c = state_c.detach()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  print({'epoch': epoch, 'loss': loss.item()})

{'epoch': 0, 'loss': 7.471439361572266}
{'epoch': 1, 'loss': 7.292314052581787}
{'epoch': 2, 'loss': 7.134233474731445}
{'epoch': 3, 'loss': 6.966627597808838}
{'epoch': 4, 'loss': 6.678420543670654}
{'epoch': 5, 'loss': 6.5347700119018555}
{'epoch': 6, 'loss': 6.365240573883057}
{'epoch': 7, 'loss': 6.23342227935791}
{'epoch': 8, 'loss': 6.073067665100098}
{'epoch': 9, 'loss': 5.977579593658447}
{'epoch': 10, 'loss': 5.8150434494018555}
{'epoch': 11, 'loss': 5.687376022338867}
{'epoch': 12, 'loss': 5.526748180389404}
{'epoch': 13, 'loss': 5.463479042053223}
{'epoch': 14, 'loss': 5.347395896911621}
{'epoch': 15, 'loss': 5.277071952819824}
{'epoch': 16, 'loss': 5.217566967010498}
{'epoch': 17, 'loss': 5.101416110992432}
{'epoch': 18, 'loss': 4.935789585113525}
{'epoch': 19, 'loss': 4.917048454284668}
{'epoch': 20, 'loss': 4.687989234924316}
{'epoch': 21, 'loss': 4.607418060302734}
{'epoch': 22, 'loss': 4.491480827331543}
{'epoch': 23, 'loss': 4.4563679695129395}
{'epoch': 24, 'loss': 4.

In [82]:
def predict(dataset, model, text, next_words=20): # 20 - необходимый размер текста
    words = text.split(' ')
    model.eval()

    state_h, state_c = model.init_state(len(words))

    for i in range(next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])
    return words


print(*predict(dataset, model, text='You'))

You enough *out*. -- Im do did to convince a eyes? I can understand. gonna repeat a mesolimbic ice sir. And
